#Pré processamento


In [ ]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
 
# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# iniciar uma sessão local e importar dados 
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace

sc = SparkSession.builder.master('local[*]').getOrCreate()
 
#df_texto_linha = sc.read.options(delimiter=';').csv("divulgacao_anos_finais_escolas_2019.csv", inferSchema=True, header=True)
df_escola = sc.read.options(delimiter='\t').csv("/content/drive/MyDrive/Poli/GRUPO_ATIVIDADES_POLI/PROJETO CIÊNCIA DE DADOS/bases tratadas/clean_divulgacao_anos_finais_escolas_2019.csv", inferSchema=True, header=True)
df_aluno_17 = sc.read.options(delimiter=',').csv("/content/drive/MyDrive/Poli/GRUPO_ATIVIDADES_POLI/PROJETO CIÊNCIA DE DADOS/SAEB 2017/TS_ALUNO_9EF.csv", inferSchema=True, header=True)
df_aluno_15 = sc.read.options(delimiter=',').csv("/content/drive/MyDrive/Poli/GRUPO_ATIVIDADES_POLI/PROJETO CIÊNCIA DE DADOS/SAEB 2015/TS_ALUNO_9EF.csv", inferSchema=True, header=True)

In [ ]:
df_join_17 = df_aluno_17.join(df_escola, df_escola.cod_escola == df_aluno_17.ID_ESCOLA, 'inner')
#df_join_17.repartition(1).write.mode('overwrite').option("sep","\t").option("header", "true").csv('/content/drive/MyDrive/Poli/GRUPO_ATIVIDADES_POLI/PROJETO CIÊNCIA DE DADOS/bases tratadas/join_aluno_escola_saeb_2017')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as st

## Base SAEP alunos nono ano 2017

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#df = pd.read_csv("/content/drive/MyDrive/UPE/Projeto Ciência de Dados/dataset/join_aluno_escola_saeb_2017.csv",  sep='\t', decimal=".")
#df.head(3)

In [ ]:
df_selecionadas_2017 = df_join_17[['ID_ALUNO','ID_REGIAO', 'ID_UF', 'ID_AREA', 'ID_DEPENDENCIA_ADM','ID_LOCALIZACAO', 'ID_TURNO',
                            'PROFICIENCIA_MT_SAEB', 'PROFICIENCIA_LP_SAEB',
                            'TX_RESP_Q002',
                            'TX_RESP_Q018',
                            'TX_RESP_Q022',
                            'TX_RESP_Q019',
                            'TX_RESP_Q023',
                            'TX_RESP_Q031',
                            'TX_RESP_Q027',
                            'TX_RESP_Q028',
                            'TX_RESP_Q030',
                            'TX_RESP_Q026',
                            'TX_RESP_Q017',
                            'TX_RESP_Q013',
                            'TX_RESP_Q011',
                            'TX_RESP_Q046',
                            'TX_RESP_Q048',
                            'TX_RESP_Q049',
                            'TX_RESP_Q044',
                            'TX_RESP_Q045',
                            'TX_RESP_Q033',
                            'TX_RESP_Q036', 'TX_RESP_Q057']]
#df_selecionadas_2017


In [ ]:
df_selecionadas_2017["PROFICIENCIA_MEDIA"] = df_selecionadas_2017[["PROFICIENCIA_MT_SAEB", "PROFICIENCIA_LP_SAEB"]].mean(axis = 1)

## Retirando valores nulos na proeficiencia (significa que o aluno faltou a prova)

In [ ]:
##Valores NA##
#df_selecionadas_2017.isnull().sum()


In [ ]:
df_selecionadas_2017 = df_selecionadas_2017.dropna(subset=["PROFICIENCIA_MEDIA"])


In [ ]:
#df_selecionadas_2017.isnull().sum()

## Filtros de remocao 

In [ ]:
# removendo as escolas federais da base

print(df_selecionadas_2017.groupby('ID_DEPENDENCIA_ADM').size())

filtro = df_selecionadas_2017['ID_DEPENDENCIA_ADM'] != 1

df_selecionadas_2017 = df_selecionadas_2017[filtro]

print(df_selecionadas_2017.groupby('ID_DEPENDENCIA_ADM').size())

In [ ]:
# retirada do turno da noite

print(df_selecionadas_2017.groupby('ID_TURNO').size())

filtro = df_selecionadas_2017['ID_TURNO'] != 3

df_selecionadas_2017 = df_selecionadas_2017[filtro]

print(df_selecionadas_2017.groupby('ID_TURNO').size())



## Preenchendo NA com a moda

In [ ]:
df_selecionadas_2017.isnull().sum()

In [ ]:
moda=st.mode(df_selecionadas_2017['TX_RESP_Q002'])
df_selecionadas_2017['TX_RESP_Q002']=df_selecionadas_2017['TX_RESP_Q002'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q018'])
df_selecionadas_2017['TX_RESP_Q018']=df_selecionadas_2017['TX_RESP_Q018'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q022'])
df_selecionadas_2017['TX_RESP_Q022']=df_selecionadas_2017['TX_RESP_Q022'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q019'])
df_selecionadas_2017['TX_RESP_Q019']=df_selecionadas_2017['TX_RESP_Q019'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q023'])
df_selecionadas_2017['TX_RESP_Q023']=df_selecionadas_2017['TX_RESP_Q023'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q031'])
df_selecionadas_2017['TX_RESP_Q031']=df_selecionadas_2017['TX_RESP_Q031'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q027'])
df_selecionadas_2017['TX_RESP_Q027']=df_selecionadas_2017['TX_RESP_Q027'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q028'])
df_selecionadas_2017['TX_RESP_Q028']=df_selecionadas_2017['TX_RESP_Q028'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q030'])
df_selecionadas_2017['TX_RESP_Q030']=df_selecionadas_2017['TX_RESP_Q030'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q026'])
df_selecionadas_2017['TX_RESP_Q026']=df_selecionadas_2017['TX_RESP_Q026'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q017'])
df_selecionadas_2017['TX_RESP_Q017']=df_selecionadas_2017['TX_RESP_Q017'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q013'])
df_selecionadas_2017['TX_RESP_Q013']=df_selecionadas_2017['TX_RESP_Q013'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q011'])
df_selecionadas_2017['TX_RESP_Q011']=df_selecionadas_2017['TX_RESP_Q011'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q046'])
df_selecionadas_2017['TX_RESP_Q046']=df_selecionadas_2017['TX_RESP_Q046'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q048'])
df_selecionadas_2017['TX_RESP_Q048']=df_selecionadas_2017['TX_RESP_Q048'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q049'])
df_selecionadas_2017['TX_RESP_Q049']=df_selecionadas_2017['TX_RESP_Q049'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q044'])
df_selecionadas_2017['TX_RESP_Q044']=df_selecionadas_2017['TX_RESP_Q044'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q045'])
df_selecionadas_2017['TX_RESP_Q045']=df_selecionadas_2017['TX_RESP_Q045'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q033'])
df_selecionadas_2017['TX_RESP_Q033']=df_selecionadas_2017['TX_RESP_Q033'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q036'])
df_selecionadas_2017['TX_RESP_Q036']=df_selecionadas_2017['TX_RESP_Q036'].fillna(moda)

moda=st.mode(df_selecionadas_2017['TX_RESP_Q057'])
df_selecionadas_2017['TX_RESP_Q057']=df_selecionadas_2017['TX_RESP_Q057'].fillna(moda)



In [ ]:
df_selecionadas_2017.isnull().sum()

## Dicotomizando as variáveis da base de 2017 (passando para numérica)

In [ ]:
##Codificando as variáveis##

#norte-nordeste - 0
#sul-sudeste-centro - 1

print(df_selecionadas_2017.groupby('ID_REGIAO').size())

df_selecionadas_2017['ID_REGIAO']=np.where(
    df_selecionadas_2017['ID_REGIAO']==1,0,
        np.where(df_selecionadas_2017['ID_REGIAO']==2,0,1))

                
print(df_selecionadas_2017.groupby('ID_REGIAO').size())

In [ ]:
##Codificando as variáveis##

df_selecionadas_2017.groupby(by = 'TX_RESP_Q031').count()
print(df_selecionadas_2017.groupby('TX_RESP_Q031').size())

df_selecionadas_2017['TX_RESP_Q031']=np.where(df_selecionadas_2017['TX_RESP_Q031']=='A',1,0)
print(df_selecionadas_2017.groupby('TX_RESP_Q031').size())


In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q018').size())
df_selecionadas_2017['TX_RESP_Q018']=np.where(df_selecionadas_2017['TX_RESP_Q018']=='B',0,1)
print(df_selecionadas_2017.groupby('TX_RESP_Q018').size())


In [ ]:

print(df_selecionadas_2017.groupby('TX_RESP_Q022').size())
df_selecionadas_2017['TX_RESP_Q022']=np.where(df_selecionadas_2017['TX_RESP_Q022']=='B',0,1)
print(df_selecionadas_2017.groupby('TX_RESP_Q022').size())


In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q027').size())
df_selecionadas_2017['TX_RESP_Q027']=np.where(df_selecionadas_2017['TX_RESP_Q027']=='B',0,1)
print(df_selecionadas_2017.groupby('TX_RESP_Q027').size())


In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q028').size())
df_selecionadas_2017['TX_RESP_Q028']=np.where(df_selecionadas_2017['TX_RESP_Q028']=='B',0,1)
print(df_selecionadas_2017.groupby('TX_RESP_Q028').size())



In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q030').size())
df_selecionadas_2017['TX_RESP_Q030']=np.where(df_selecionadas_2017['TX_RESP_Q030']=='B',0,1)
print(df_selecionadas_2017.groupby('TX_RESP_Q030').size())



In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q011').size())

df_selecionadas_2017['TX_RESP_Q011']=np.where(df_selecionadas_2017['TX_RESP_Q011']=='A',0,1)
print(df_selecionadas_2017.groupby('TX_RESP_Q011').size())

In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q036').size())

df_selecionadas_2017['TX_RESP_Q036']=np.where(
	df_selecionadas_2017['TX_RESP_Q036']=='A',2,
	np.where(
		df_selecionadas_2017['TX_RESP_Q036']=='B',1
		,np.where(
			df_selecionadas_2017['TX_RESP_Q036']=='C',0,0
		  )
			)
			)
                
print(df_selecionadas_2017.groupby('TX_RESP_Q036').size())

In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q033').size())

df_selecionadas_2017['TX_RESP_Q033']=np.where(
	df_selecionadas_2017['TX_RESP_Q033']=='A',2,
	np.where(
		df_selecionadas_2017['TX_RESP_Q033']=='B',1
		,np.where(
			df_selecionadas_2017['TX_RESP_Q033']=='C',0,0
		  )
			)
			)
                
print(df_selecionadas_2017.groupby('TX_RESP_Q033').size())

In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q045').size())

df_selecionadas_2017['TX_RESP_Q045']=np.where(df_selecionadas_2017['TX_RESP_Q045']=='A',1,0)
print(df_selecionadas_2017.groupby('TX_RESP_Q045').size())

In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q044').size())

df_selecionadas_2017['TX_RESP_Q044']=np.where(df_selecionadas_2017['TX_RESP_Q044']=='E',0,1)
print(df_selecionadas_2017.groupby('TX_RESP_Q044').size())

In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q013').size())
df_selecionadas_2017['TX_RESP_Q013']=np.where(df_selecionadas_2017['TX_RESP_Q013']=='A',0,1)
print(df_selecionadas_2017.groupby('TX_RESP_Q013').size())



In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q048').size())

df_selecionadas_2017['TX_RESP_Q048']=np.where(
	df_selecionadas_2017['TX_RESP_Q048']=='A',0,
	np.where(
		df_selecionadas_2017['TX_RESP_Q048']=='B',1
		,np.where(
			df_selecionadas_2017['TX_RESP_Q048']=='C',2,2
		  )
			)
			)
                
print(df_selecionadas_2017.groupby('TX_RESP_Q048').size())

In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q049').size())
df_selecionadas_2017['TX_RESP_Q049']=np.where(df_selecionadas_2017['TX_RESP_Q049']=='A',0,1)
print(df_selecionadas_2017.groupby('TX_RESP_Q049').size())

In [ ]:
# A = 0 B = 0, C = 1, D = 2, E = 3, F = 4
# G = 5 , null = None

print(df_selecionadas_2017.groupby('TX_RESP_Q019').size())

df_selecionadas_2017['TX_RESP_Q019']=np.where(
	df_selecionadas_2017['TX_RESP_Q019']=='A',0,
	np.where(
		df_selecionadas_2017['TX_RESP_Q019']=='B',0
		,np.where(
			df_selecionadas_2017['TX_RESP_Q019']=='C',1,
			np.where(
				df_selecionadas_2017['TX_RESP_Q019']=='D',2,
				np.where(
					df_selecionadas_2017['TX_RESP_Q019']=='E',3,
					np.where(
						df_selecionadas_2017['TX_RESP_Q019']=='F',4,
						np.where(
							df_selecionadas_2017['TX_RESP_Q019']=='G',5 ,5
			)
			)
			)))))
                
print(df_selecionadas_2017.groupby('TX_RESP_Q019').size())

# qtd de nulos 29480

In [ ]:
# A = 0, B = 0, C = 1, D = 2, E = 3, F = 4 
# G null


print(df_selecionadas_2017.groupby('TX_RESP_Q023').size())

df_selecionadas_2017['TX_RESP_Q023']=np.where(
	df_selecionadas_2017['TX_RESP_Q023']=='A',0,
	np.where(
		df_selecionadas_2017['TX_RESP_Q023']=='B',0
		,np.where(
			df_selecionadas_2017['TX_RESP_Q023']=='C',1,
			np.where(
				df_selecionadas_2017['TX_RESP_Q023']=='D',2,
				np.where(
					df_selecionadas_2017['TX_RESP_Q023']=='E',3,
					np.where(
						df_selecionadas_2017['TX_RESP_Q023']=='F',4,
						np.where(
							df_selecionadas_2017['TX_RESP_Q023']=='G', 5 ,5
			)
			)
			)))))
                
print(df_selecionadas_2017.groupby('TX_RESP_Q023').size())



#df_selecionadas_2017['TX_RESP_Q023']=np.where(df_selecionadas_2017['TX_RESP_Q023']=='A',0,1)
#print(df_selecionadas_2017.groupby('TX_RESP_Q023').size())

In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q026').size())

df_selecionadas_2017['TX_RESP_Q026']=np.where(
	df_selecionadas_2017['TX_RESP_Q026']=='A',2,
	np.where(
		df_selecionadas_2017['TX_RESP_Q026']=='B',1
		,np.where(
			df_selecionadas_2017['TX_RESP_Q026']=='C',0,0
		  )
			)
			)
print(df_selecionadas_2017.groupby('TX_RESP_Q026').size())

In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q017').size())
df_selecionadas_2017['TX_RESP_Q017']=np.where(df_selecionadas_2017['TX_RESP_Q017']=='A',0,1)
print(df_selecionadas_2017.groupby('TX_RESP_Q017').size())

In [ ]:
# A = 0, B = 1, C = 2 e D = 3


print(df_selecionadas_2017.groupby('TX_RESP_Q046').size())

df_selecionadas_2017['TX_RESP_Q046']=np.where(
	df_selecionadas_2017['TX_RESP_Q046']=='A',0,
	np.where(
		df_selecionadas_2017['TX_RESP_Q046']=='B',1
		,np.where(
			df_selecionadas_2017['TX_RESP_Q046']=='C',2,
			np.where(
				df_selecionadas_2017['TX_RESP_Q046']=='D',3, 3
			)
			)
			))
                
print(df_selecionadas_2017.groupby('TX_RESP_Q046').size())

In [ ]:
print(df_selecionadas_2017.groupby('TX_RESP_Q057').size())

df_selecionadas_2017['TX_RESP_Q057']=np.where(
	df_selecionadas_2017['TX_RESP_Q057']=='A',0,
	np.where(
		df_selecionadas_2017['TX_RESP_Q057']=='B',2
		,np.where(
			df_selecionadas_2017['TX_RESP_Q057']=='C',1,
			np.where(
				df_selecionadas_2017['TX_RESP_Q057']=='D',3,4
			)
			)
			))
                
print(df_selecionadas_2017.groupby('TX_RESP_Q057').size())


## Renomeando variaveis

In [ ]:
df_selecionadas_2017 = df_selecionadas_2017.rename(columns={"ID_REGIAO": "REGIAO",
                                     "ID_UF": "UF", "ID_AREA": "AREA", 
                                     "ID_DEPENDENCIA_ADM": "DEPENDENCIA_ADM", 
                                     "ID_LOCALIZACAO": "LOCALIZACAO", "ID_TURNO": "TURNO",
                                     'TX_RESP_Q002': "COR_RACA", # nao converter, pois n entra no modelo
                                     'TX_RESP_Q018': "MORA_MAE", 
                                     'TX_RESP_Q022': "MORA_PAI",
                                     'TX_RESP_Q019': "ESCOLARIDADE_MAE", #
                                     'TX_RESP_Q023': "ESCOLARIDADE_PAI", # 
                                     'TX_RESP_Q031': "CONVERSA_PAIS_SOBRE_ESCOLA", 
                                     'TX_RESP_Q027': "INCENTIVO_PAIS_ESTUDAR",
                                     'TX_RESP_Q028': "INCENTIVO_PAIS_DEVER_DE_CASA", 
                                     'TX_RESP_Q030': "INCENTIVO_PAIS_IR_ESCOLA", 
                                     'TX_RESP_Q026': "IDA_PAIS_REUNIOES", #
                                     'TX_RESP_Q017': "TEM_EMPREGADA_O", #
                                     'TX_RESP_Q013': "TEM_COMPUTADOR", #
                                     'TX_RESP_Q011': "TEM_MAQUINA_LAVAR_ROUPAS",
                                     'TX_RESP_Q046': "QUANDO_ENTROU_ESCOLA", #
                                     'TX_RESP_Q048': "JA_FOI_REPROVADO", #
                                     'TX_RESP_Q049': "ABANDONOU_ESCOLA", #
                                     'TX_RESP_Q044': "TEMPO_GASTO_TRAB_DOMESTICO",#
                                     'TX_RESP_Q045': "TRABALHO_EXTERNO", #
                                     'TX_RESP_Q033': "LE_LIVROS",#
                                     'TX_RESP_Q036': "LE_GIBI",#
                                     'TX_RESP_Q057' : "OQUE_FAZER_POS_FUNDAMENTAL"})

df_selecionadas_2017.columns

In [ ]:
df_selecionadas_2017.describe().round(2)

In [ ]:
#df_sel_tx = df_selecionadas_2017.filter(like='TX_RESP_Q', axis=1)
#df_sel_tx.columns

In [ ]:
# OBETIVO: Prever a proeficiencia do aluno com as variaveis. 

## como selecionar as variaveis? 
### correlação de Pearson

In [ ]:
## tirar Nan das variaveis de proeficiencia de port e mate (pois foram alunos que faltaram a prova)

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html

# df.dropna(subset=['name', 'toy'])

In [ ]:
## calcular a media das vars de proeficiencia (ok)

In [ ]:
## converter as vars de A,B,C,D para dicotomica 0 ou 1.

In [ ]:
## ver medidas de associação para as vars e proeficiencia (qui-quadrado por exemplo)

## Salvando base filtrada

In [ ]:
df_selecionadas_2017.to_csv("base_2017_filtrada_v3_SEM_NA.csv", sep='\t')

In [ ]:
df_selecionadas_2017.to_csv("/content/drive/MyDrive/UPE/Projeto Ciência de Dados/dataset/base_2017_filtrada_v3_SEM_NA.csv", sep='\t')

In [ ]:
# Base de dados para modelagem 

In [ ]:
df_2017 = pd.read_csv('./dataset/base_2017_filtrada.csv',  sep='\t', decimal=".")
df_2017.head(5)

In [ ]:
df_selecionadas_2017.isnull().sum()

In [ ]:
df_selecionadas_2017.groupby(by = 'COR_RACA').count()

In [ ]:
df_selecionadas_2017.head()

## Base SAEP alunos nono ano 2019

In [ ]:
df_2019 = pd.read_csv('./dataset/join_aluno_escola_saeb_2019.csv',  sep='\t', decimal=".")
df_2019

In [ ]:
df_sel_tx = df_2019.filter(like='TX_RESP_Q0', axis=1)
df_sel_tx.columns

In [ ]:
df_selecionadas_2019 = df_2019[['ID_ALUNO','ID_REGIAO', 'ID_UF', 'ID_AREA', 'ID_DEPENDENCIA_ADM','ID_LOCALIZACAO', 'ID_TURNO',
                            'PROFICIENCIA_MT_SAEB', 'PROFICIENCIA_LP_SAEB',
                            'TX_RESP_Q002', #"COR_RACA", 
                            'TX_RESP_Q003A', #"MORA_MAE", 
                            'TX_RESP_Q003B', #"MORA_PAI",
                            'TX_RESP_Q004', #"ESCOLARIDADE_MAE", 
                            'TX_RESP_Q005', #"ESCOLARIDADE_PAI",
                            'TX_RESP_Q006A', #"CONVERSA_PAIS_SOBRE_ESCOLA", 
                            'TX_RESP_Q006B', #"INCENTIVO_PAIS_ESTUDAR",      
                            'TX_RESP_Q006C', #"INCENTIVO_PAIS_DEVER_DE_CASA"
                            'TX_RESP_Q006D', #"INCENTIVO_PAIS_IR_ESCOLA",  
                            'TX_RESP_Q006E', #"IDA_PAIS_REUNIOES",     
                            'TX_RESP_Q007', #"TEM_EMPREGADA_O",                                
                            'TX_RESP_Q009C', #"TEM_COMPUTADOR",                                
                            'TX_RESP_Q010H', #"TEM_MAQUINA_LAVAR_ROUPAS",                                
                            'TX_RESP_Q013', #"QUANDO_ENTROU_ESCOLA",                                 
                            'TX_RESP_Q015', #"JA_FOI_REPROVADO",                                
                            'TX_RESP_Q016', #"ABANDONOU_ESCOLA",                                
                            'TX_RESP_Q017C', #"TEMPO_GASTO_TRAB_DOMESTICO",  
                            'TX_RESP_Q017E', #"TRABALHO_EXTERNO",                                 
                            'TX_RESP_Q018B', #"LE_LIVROS",
                            'TX_RESP_Q018C', #"LE_GIBI"})
                            'TX_RESP_Q019']] #OQUE_FAZER_POS_FUNDAMENTAL
df_selecionadas_2019


# Análise Descritiva

### REGIÃO

In [ ]:
região = df_filter.groupby(by = ['ID_REGIAO'], dropna=False).count()
região = região[['ID_ALUNO']]
região['regiao'] = ['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste']


In [ ]:
região

In [ ]:
regiã0 = região.sort_values('ID_ALUNO')

In [ ]:
região.plot(x = 'regiao', y = 'ID_ALUNO',  kind='bar',
        alpha=0.9,
        rot=45,
        grid=True,
        figsize=(10,5),
        fontsize=12, 
        color=['#33513bff'], use_index  = True, label = 'Alunos')
plt.xlabel('Regiões do Brasil')
plt.ylabel("Contagem ID_ALUNO")

### UF

In [ ]:
uf = df_filter.groupby(by = ['ID_UF'], dropna=False).count()
uf = uf[['ID_ALUNO']] #grafico de barras invertido

In [ ]:
uf.plot(y = 'ID_ALUNO',  kind='barh',
        alpha=0.9,
        rot=0,
        grid=True,
        figsize=(9,6),
        fontsize=12, 
        color=['#33513bff'], use_index  = False)
plt.xlabel('ufd do Brasil')
plt.ylabel("Contagem ID_ALUNO")

### AREA

In [ ]:
area = df_filter.groupby(by = ['ID_AREA'], dropna=False).count()
area = area['ID_ALUNO']  #Grafico de setores

#1 - Capital
#2 - Interior

In [ ]:
area

In [ ]:

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Interior','Capital'
sizes = [81.4, 18.6]
explode = (0, 0.1,)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()

ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)

ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
# Cada plot terá o mesmo tamanho de figuras (10,5)
%matplotlib inline
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3, ax4) = plt.subplots(2, 2, figsize=(10,5))


import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
x = df_filter["tx_aprov_09_6a9"]
plt.hist(x, density=True, bins=30)  # `density=False` would make counts
plt.ylabel('Frequência')
plt.xlabel('Taxa de Aprovação - Ano de 2009');
plt.title("Histograma da Taxa de Aprovação do 6° ao 9° ano - Ano de 2009");

## Análise Descritiva

##**Ano 2005** 

In [ ]:
df_filter[["tx_aprov_05_6a9", "nt_saeb_05_nmp", "ideb_05_np"]].describe()

O Censo escolar no ano de 2005 apresenta taxa de aprovação média de 74%. Foi observada que a avaliação média das escolas era de 4.3. Como esse é o primeiro ano da base, não é informada a projeção de nota esperada.

In [ ]:
# Define as configurações dos plots
# Cada plot terá o mesmo tamanho de figuras (10,5)
%matplotlib inline

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))


x1 = df["tx_aprov_05_6a9"]
x2 = df["nt_saeb_05_nmp"]
x3 = df["ideb_05_np"]


# Dados para cada subplot
ax1.hist(x1, density=True, bins=30,color='#3DB2F2')
ax2.hist(x2, density=True, bins=30, color='#62A6F5')
ax3.hist(x3, density=True, bins=30,color='#2A5EE0')


ax1.set(title="Taxa de Aprovação (2005)", xlabel="Eixo x", ylabel=" Eixo y")
ax2.set(title="Notas SAEB (2005)", xlabel="Eixo x", ylabel=" Eixo y")
ax3.set(title="IDEB (2005)", xlabel="Eixo x", ylabel=" Eixo y")


##**Ano 2007** 

In [ ]:
df_filter[["tx_aprov_07_6a9", "nt_saeb_07_nmp", "ideb_07_np", "proj_07"]].describe()

O Censo escolar no ano de 2007 apresenta taxa de aprovação média de 79%. A projeção para o ano é de cerca de 3.3 pontos onde foi observada uma avaliação média de 4.4. Ou seja, de forma geral a expectativa foi superada, dado que as métricas reais são superiores as esperadas.

In [ ]:
# Define as configurações dos plots
# Cada plot terá o mesmo tamanho de figuras (10,5)
%matplotlib inline

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15,5))


x1 = df["tx_aprov_07_6a9"]
x2 = df["nt_saeb_07_nmp"]
x3 = df["ideb_07_np"]
x4 = df["proj_07"]


# Dados para cada subplot
ax1.hist(x1, density=True, bins=30,color='#3DB2F2')
ax2.hist(x2, density=True, bins=30, color='#62A6F5')
ax3.hist(x3, density=True, bins=30,color='#2A5EE0')
ax4.hist(x4, density=True, bins=30, color='#2339F7')

ax1.set(title="Taxa de Aprovação (2007)", xlabel="Eixo x", ylabel=" Eixo y")
ax2.set(title="Notas SAEB (2007)", xlabel="Eixo x", ylabel=" Eixo y")
ax3.set(title="IDEB (2007)", xlabel="Eixo x", ylabel=" Eixo y")
ax4.set(title="Projeção (2007)", xlabel="Eixo x", ylabel=" Eixo y")

##**Ano 2009** 

In [ ]:
df_filter[["tx_aprov_09_6a9", "nt_saeb_09_nmp", "ideb_09_np", "proj_09"]].describe()

O Censo escolar no ano de 2009 apresenta taxa de aprovação média de 80%. A projeção para o ano é de cerca de 3.5 pontos onde foi observada uma avaliação média de 4.6. Ou seja, de forma geral a expectativa foi superada, dado que as métricas reais são superiores as esperadas.

In [ ]:
# Define as configurações dos plots
# Cada plot terá o mesmo tamanho de figuras (10,5)
%matplotlib inline

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15,5))


x1 = df["tx_aprov_09_6a9"]
x2 = df["nt_saeb_09_nmp"]
x3 = df["ideb_09_np"]
x4 = df["proj_09"]


# Dados para cada subplot
ax1.hist(x1, density=True, bins=30,color='#3DB2F2')
ax2.hist(x2, density=True, bins=30, color='#62A6F5')
ax3.hist(x3, density=True, bins=30,color='#2A5EE0')
ax4.hist(x4, density=True, bins=30, color='#2339F7')

ax1.set(title="Taxa de Aprovação (2009)", xlabel="Eixo x", ylabel=" Eixo y")
ax2.set(title="Notas SAEB (2009)", xlabel="Eixo x", ylabel=" Eixo y")
ax3.set(title="IDEB (2009)", xlabel="Eixo x", ylabel=" Eixo y")
ax4.set(title="Projeção (2009)", xlabel="Eixo x", ylabel=" Eixo y")

##**Ano 2011** 

In [ ]:
df_filter[["tx_aprov_11_6a9", "nt_saeb_11_nmp", "ideb_11_np", "proj_11"]].describe()

O Censo escolar no ano de 2011 apresenta taxa de aprovação média de 82%, 2 pontos a mais que o censo anterior. A projeção para o ano é de cerca de 3.8 pontos onde foi observada uma avaliação média de 4.6. Ou seja, de forma geral a expectativa foi superada, dado que a maioria das métricas reais são superiores as esperadas.

In [ ]:
# Define as configurações dos plots
# Cada plot terá o mesmo tamanho de figuras (10,5)
%matplotlib inline

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15,5))


x1 = df["tx_aprov_11_6a9"]
x2 = df["nt_saeb_11_nmp"]
x3 = df["ideb_11_np"]
x4 = df["proj_11"]


# Dados para cada subplot
ax1.hist(x1, density=True, bins=30,color='#3DB2F2')
ax2.hist(x2, density=True, bins=30, color='#62A6F5')
ax3.hist(x3, density=True, bins=30,color='#2A5EE0')
ax4.hist(x4, density=True, bins=30, color='#2339F7')

ax1.set(title="Taxa de Aprovação (2011)", xlabel="Eixo x", ylabel=" Eixo y")
ax2.set(title="Notas SAEB (2011)", xlabel="Eixo x", ylabel=" Eixo y")
ax3.set(title="IDEB (2011)", xlabel="Eixo x", ylabel=" Eixo y")
ax4.set(title="Projeção (2011)", xlabel="Eixo x", ylabel=" Eixo y")

##**Ano 2013** 

In [ ]:
df_filter[["tx_aprov_13_6a9", "nt_saeb_13_nmp", "ideb_13_np", "proj_13"]].describe()

O Censo escolar no ano de 2013 apresenta taxa de aprovação média de 84%, 2 pontos a mais que o ano anterior. A projeção para o ano é de cerca de 4.1 pontos onde foi observada uma avaliação média de 4.6. Ou seja, de forma geral a expectativa foi superada, dado que as métricas reais são superiores as esperadas.

In [ ]:
# Define as configurações dos plots
# Cada plot terá o mesmo tamanho de figuras (10,5)
%matplotlib inline

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15,5))


x1 = df["tx_aprov_13_6a9"]
x2 = df["nt_saeb_13_nmp"]
x3 = df["ideb_13_np"]
x4 = df["proj_13"]


# Dados para cada subplot
ax1.hist(x1, density=True, bins=30,color='#3DB2F2')
ax2.hist(x2, density=True, bins=30, color='#62A6F5')
ax3.hist(x3, density=True, bins=30,color='#2A5EE0')
ax4.hist(x4, density=True, bins=30, color='#2339F7')

ax1.set(title="Taxa de Aprovação (2013)", xlabel="Eixo x", ylabel=" Eixo y")
ax2.set(title="Notas SAEB (2013)", xlabel="Eixo x", ylabel=" Eixo y")
ax3.set(title="IDEB (2013)", xlabel="Eixo x", ylabel=" Eixo y")
ax4.set(title="Projeção (2013)", xlabel="Eixo x", ylabel=" Eixo y")

##**Ano 2015** 

In [ ]:
df_filter[["tx_aprov_15_6a9", "nt_saeb_15_nmp", "ideb_15_np", "proj_15"]].describe()

O Censo escolar no ano de 2015 apresenta taxa de aprovação média de 84%, taxa similar a do último censo realizado no ano de 2013. A projeção para o ano é de cerca de 4.4 pontos onde foi observada uma avaliação média de 4.9. Ou seja, de forma geral a expectativa foi superada, dado que maior parte das métricas reais são superiores as esperadas.

In [ ]:
# Define as configurações dos plots
# Cada plot terá o mesmo tamanho de figuras (10,5)
%matplotlib inline

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15,5))


x1 = df["tx_aprov_15_6a9"]
x2 = df["nt_saeb_15_nmp"]
x3 = df["ideb_15_np"]
x4 = df["proj_15"]


# Dados para cada subplot
ax1.hist(x1, density=True, bins=30,color='#3DB2F2')
ax2.hist(x2, density=True, bins=30, color='#62A6F5')
ax3.hist(x3, density=True, bins=30,color='#2A5EE0')
ax4.hist(x4, density=True, bins=30, color='#2339F7')

ax1.set(title="Taxa de Aprovação (2015)", xlabel="Eixo x", ylabel=" Eixo y")
ax2.set(title="Notas SAEB (2015)", xlabel="Eixo x", ylabel=" Eixo y")
ax3.set(title="IDEB (2015)", xlabel="Eixo x", ylabel=" Eixo y")
ax4.set(title="Projeção (2015)", xlabel="Eixo x", ylabel=" Eixo y")

##**Ano 2017** 

In [ ]:
df_filter[["tx_aprov_17_6a9", "nt_saeb_17_nmp", "ideb_17_np", "proj_17"]].describe()

O Censo escolar no ano de 2017 apresenta taxa de aprovação média de 86%, apresenta 2 pontos a mais que o censo de 2015. A projeção para o ano é de cerca de 4.7 pontos onde foi observada uma avaliação média de 5.0. Ou seja, de forma geral a expectativa foi superada, dado que a maioria das métricas reais são superiores as esperadas.

In [ ]:
# Define as configurações dos plots
# Cada plot terá o mesmo tamanho de figuras (10,5)
%matplotlib inline

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15,5))


x1 = df["tx_aprov_17_6a9"]
x2 = df["nt_saeb_17_nmp"]
x3 = df["ideb_17_np"]
x4 = df["proj_17"]


# Dados para cada subplot
ax1.hist(x1, density=True, bins=30,color='#3DB2F2')
ax2.hist(x2, density=True, bins=30, color='#62A6F5')
ax3.hist(x3, density=True, bins=30,color='#2A5EE0')
ax4.hist(x4, density=True, bins=30, color='#2339F7')

ax1.set(title="Taxa de Aprovação (2017)", xlabel="Eixo x", ylabel=" Eixo y")
ax2.set(title="Notas SAEB (2017)", xlabel="Eixo x", ylabel=" Eixo y")
ax3.set(title="IDEB (2017)", xlabel="Eixo x", ylabel=" Eixo y")
ax4.set(title="Projeção (2017)", xlabel="Eixo x", ylabel=" Eixo y")

##**Ano 2019** 

In [ ]:
df_filter[["tx_aprov_19_6a9", "nt_saeb_19_nmp", "ideb_19_np", "proj_19"]].describe()

O Censo escolar no ano de 2019 apresenta taxa de aprovação média de 88%, seguindo o padrão observado ao longo dos anos, houve um acréscimo de dois pontos na aprovação. A projeção para o ano é de cerca de 4.9 pontos onde foi observada uma avaliação média de 5.1. Ou seja, de forma geral a expectativa foi superada, dado que grande parte das métricas reais são superiores as esperadas.

In [ ]:
# Define as configurações dos plots
# Cada plot terá o mesmo tamanho de figuras (10,5)
%matplotlib inline

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15,5))


x1 = df["tx_aprov_19_6a9"]
x2 = df["nt_saeb_19_nmp"]
x3 = df["ideb_19_np"]
x4 = df["proj_19"]


# Dados para cada subplot
ax1.hist(x1, density=True, bins=30,color='#3DB2F2')
ax2.hist(x2, density=True, bins=30, color='#62A6F5')
ax3.hist(x3, density=True, bins=30,color='#2A5EE0')
ax4.hist(x4, density=True, bins=30, color='#2339F7')

ax1.set(title="Taxa de Aprovação (2019)", xlabel="Eixo x", ylabel=" Eixo y")
ax2.set(title="Notas SAEB (2019)", xlabel="Eixo x", ylabel=" Eixo y")
ax3.set(title="IDEB (2019)", xlabel="Eixo x", ylabel=" Eixo y")
ax4.set(title="Projeção (2019)", xlabel="Eixo x", ylabel=" Eixo y")

##**Ano 2021** 

In [ ]:
df_filter[["proj_21"]].describe()

Para o ano de 2021, a projeção média das notas esperadas é de 5.13.  

In [ ]:
# Define as configurações dos plots
# Cada plot terá o mesmo tamanho de figuras (10,5)
%matplotlib inline

fig, (ax4) = plt.subplots(1, 1, figsize=(15,5))


x4 = df["proj_21"]


# Dados para cada subplot
ax4.hist(x4, density=True, bins=30, color='#2339F7')

ax4.set(title="Projeção (2021)", xlabel="Eixo x", ylabel=" Eixo y")

Fazendo uma análise descritiva ao longo dos anos, observamos que apesar de sempre apresentar um leve acréscimo no valor médio das variáveis, não há grandes mudanças no comportamento comparando ano a ano. Observamos também que nenhuma escola obteve nota nas extremidades (0 ou 10) ao decorrer dos anos.